# Data Cleaning: Flights table

In [1]:
# Import packages
import pandas as pd
import numpy as np
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine
from datetime import datetime, date, timedelta

In [2]:
# Allows notebook to display output from multiple lines in the same cess
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

[Source](https://blog.panoply.io/connecting-jupyter-notebook-with-postgresql-for-python-data-analysis) for database connection code

In [3]:
# import username and password
from my_access import lhl_pg
user = lhl_pg['username']
pword = lhl_pg['password']

In [4]:
# Postgres username, password, and database name
POSTGRES_ADDRESS = 'mid-term-project.ca2jkepgjpne.us-east-2.rds.amazonaws.com' 
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = user
POSTGRES_PASSWORD = pword
POSTGRES_DBNAME = 'mid_term_project' ## CHANGE THIS TO YOUR DATABASE NAME

postgres_str = (f'postgresql://{user}:{pword}@{POSTGRES_ADDRESS}:{POSTGRES_PORT}/{POSTGRES_DBNAME}')

cnx = create_engine(postgres_str)

In [5]:
# Import first 10,000 lines form the flights table

# query = "Select * FROM flights LIMIT 20000;"
# df = pd.read_sql_query(query, cnx)

In [6]:
# Export df to csv for later retrieval
# df.to_csv('flights_sample.csv')

In [7]:
# # Import random 10,000 lines form the flights table

# query = "Select * FROM flights ORDER BY random() LIMIT 10000;"
# df_rand = pd.read_sql_query(query, cnx)
# df_rand.to_csv('flights_sample_random.csv')

In [8]:
df = pd.read_csv('flights_sample_random.csv').drop(columns = 'Unnamed: 0')
df.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,2018-08-15,DL,DL,DL,2134,DL,N942AT,2134,11292,DEN,...,1024.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-11,AA,AA,AA,457,AA,N185UW,457,14100,PHL,...,449.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-07-29,DL,DL_CODESHARE,DL,3488,9E,N314PQ,3488,11423,DSM,...,232.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-06-07,DL,DL_CODESHARE,DL,7363,OO,N900EV,7363,10141,ABR,...,257.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-07-22,UA,UA,UA,2048,UA,N62892,2048,12892,LAX,...,236.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


First off, I want to keep the rows with cancellations. But they need to be treated differently because their null values are probably meaningful. So I'll partition them and deal with them separately.

In [9]:
df_can = df[df.cancelled == 1]
df = df[df.cancelled != 1]

In [10]:
# look at data types
df.dtypes

fl_date                 object
mkt_unique_carrier      object
branded_code_share      object
mkt_carrier             object
mkt_carrier_fl_num       int64
op_unique_carrier       object
tail_num                object
op_carrier_fl_num        int64
origin_airport_id        int64
origin                  object
origin_city_name        object
dest_airport_id          int64
dest                    object
dest_city_name          object
crs_dep_time             int64
dep_time               float64
dep_delay              float64
taxi_out               float64
wheels_off             float64
wheels_on              float64
taxi_in                float64
crs_arr_time             int64
arr_time               float64
arr_delay              float64
cancelled              float64
cancellation_code       object
diverted               float64
dup                     object
crs_elapsed_time       float64
actual_elapsed_time    float64
air_time               float64
flights                float64
distance

# Filling null values

In [11]:
# list columns with null values
nan_cols = [col for col in df.columns if df[col].isnull().sum() > 0]
df[nan_cols].isnull().sum()

dep_delay                 1
taxi_out                  4
wheels_off                4
wheels_on                11
taxi_in                  11
arr_time                  7
arr_delay                34
cancellation_code      9830
actual_elapsed_time      34
air_time                 38
carrier_delay          8040
weather_delay          8040
nas_delay              8040
security_delay         8040
late_aircraft_delay    8040
first_dep_time         9776
total_add_gtime        9776
longest_add_gtime      9776
no_name                9830
dtype: int64

### `tail_num`

In [12]:
# tail_num probably isn't important, so drop it. Also dropping "No_name" and "cancellation_code"
dropcols = ['tail_num', 'no_name']

### `dep_time`, `dep_delay` etc.

In [13]:
def whatsleft(df):
    nan_cols = [col for col in df.columns if df[col].isnull().sum() > 0]
    return df[nan_cols].isnull().sum()

In [14]:
whatsleft(df)

dep_delay                 1
taxi_out                  4
wheels_off                4
wheels_on                11
taxi_in                  11
arr_time                  7
arr_delay                34
cancellation_code      9830
actual_elapsed_time      34
air_time                 38
carrier_delay          8040
weather_delay          8040
nas_delay              8040
security_delay         8040
late_aircraft_delay    8040
first_dep_time         9776
total_add_gtime        9776
longest_add_gtime      9776
no_name                9830
dtype: int64

## `Taxi out` etc.

In [15]:
# let's look at arr_delay
df[df.arr_delay.isnull()].iloc[:20,14:-9]

,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance
244,950,957.0,7.0,19.0,1016.0,NaN,NaN,1300,NaN,NaN,0.0,NaN,1.0,N,130.0,NaN,NaN,1.0,737.0
581,1615,1617.0,2.0,25.0,1642.0,2226.0,6.0,1941,2232.0,NaN,0.0,NaN,1.0,N,146.0,NaN,NaN,1.0,844.0
855,1332,1328.0,-4.0,16.0,1344.0,1940.0,3.0,1428,1943.0,NaN,0.0,NaN,1.0,N,116.0,NaN,NaN,1.0,604.0
927,1529,1521.0,-8.0,12.0,1533.0,1955.0,9.0,1740,2004.0,NaN,0.0,NaN,1.0,N,131.0,NaN,NaN,1.0,659.0
1377,710,705.0,-5.0,22.0,727.0,1623.0,5.0,1446,1628.0,NaN,0.0,NaN,1.0,N,276.0,NaN,NaN,1.0,2139.0
1522,710,719.0,9.0,9.0,728.0,NaN,NaN,1000,NaN,NaN,0.0,NaN,1.0,N,110.0,NaN,NaN,1.0,628.0
1706,850,847.0,-3.0,27.0,914.0,1415.0,6.0,1056,1421.0,NaN,0.0,NaN,1.0,N,246.0,NaN,NaN,1.0,1399.0
2072,1519,1609.0,50.0,9.0,1618.0,1953.0,44.0,1709,2037.0,NaN,0.0,NaN,1.0,N,110.0,NaN,NaN,1.0,460.0
2405,2045,2222.0,97.0,9.0,2231.0,NaN,NaN,2240,NaN,NaN,0.0,NaN,1.0,N,115.0,NaN,NaN,1.0,623.0
3274,1619,1612.0,-7.0,29.0,1641.0,NaN,NaN,1840,NaN,NaN,0.0,NaN,1.0,N,141.0,NaN,NaN,1.0,802.0


In [16]:
# we can easily fill in arr_delay with the difference between crs_arr_time and arr_time.
# But let's remove rows where both arr_delay and arr_time are missing

c1 = df.arr_time.notnull()
c2 = df.arr_delay.notnull()
df = df[c1 & c2]

In [17]:
whatsleft(df)

dep_delay                 1
taxi_out                  4
wheels_off                4
wheels_on                 4
taxi_in                   4
cancellation_code      9796
air_time                  4
carrier_delay          8006
weather_delay          8006
nas_delay              8006
security_delay         8006
late_aircraft_delay    8006
first_dep_time         9743
total_add_gtime        9743
longest_add_gtime      9743
no_name                9796
dtype: int64

Let's deal with these `*_delay` columns. Can we replace these with zeros?

In [18]:
df[df['carrier_delay'].isnull()].iloc[:,14:]

,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,700,656.0,-4.0,16.0,712.0,837.0,10.0,900,847.0,-13.0,...,1024.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1520,1534.0,14.0,19.0,1553.0,1716.0,6.0,1712,1722.0,10.0,...,449.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,801,757.0,-4.0,14.0,811.0,856.0,16.0,920,912.0,-8.0,...,232.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1319,1329.0,10.0,24.0,1353.0,1437.0,7.0,1432,1444.0,12.0,...,257.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1905,1902.0,-3.0,14.0,1916.0,2000.0,6.0,2020,2006.0,-14.0,...,236.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2130,2141.0,11.0,20.0,2201.0,525.0,5.0,525,530.0,5.0,...,2342.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9996,2000,1958.0,-2.0,19.0,2017.0,1957.0,6.0,2015,2003.0,-12.0,...,237.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9997,1318,1318.0,0.0,11.0,1329.0,1646.0,15.0,1724,1701.0,-23.0,...,1118.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9998,720,723.0,3.0,11.0,734.0,1035.0,7.0,1040,1042.0,2.0,...,957.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Yes, probably

In [19]:
delay_cols = ['carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay']
for col in delay_cols:
    df[col] = df[col].fillna(value=0)

In [20]:
whatsleft(df)

dep_delay               1
taxi_out                4
wheels_off              4
wheels_on               4
taxi_in                 4
cancellation_code    9796
air_time                4
first_dep_time       9743
total_add_gtime      9743
longest_add_gtime    9743
no_name              9796
dtype: int64

Now the last three columns. These all pertain only to cancelled or gate return flights. Can we replace null values with zero or remove the columns altogether?

In [21]:
dropcols += ['first_dep_time', 'total_add_gtime',
       'longest_add_gtime']

In [22]:
whatsleft(df)

dep_delay               1
taxi_out                4
wheels_off              4
wheels_on               4
taxi_in                 4
cancellation_code    9796
air_time                4
first_dep_time       9743
total_add_gtime      9743
longest_add_gtime    9743
no_name              9796
dtype: int64

In [23]:
# let's fill cancellation code nulls with zeros
df['cancellation_code'] = df['cancellation_code'].fillna(value='0')

In [24]:
# And drop listed columns
df = df.drop(columns=dropcols)
df.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,...,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2018-08-15,DL,DL,DL,2134,DL,2134,11292,DEN,"Denver, CO",...,180.0,171.0,145.0,1.0,1024.0,0.0,0.0,0.0,0.0,0.0
1,2018-01-11,AA,AA,AA,457,AA,457,14100,PHL,"Philadelphia, PA",...,112.0,108.0,83.0,1.0,449.0,0.0,0.0,0.0,0.0,0.0
2,2019-07-29,DL,DL_CODESHARE,DL,3488,9E,3488,11423,DSM,"Des Moines, IA",...,79.0,75.0,45.0,1.0,232.0,0.0,0.0,0.0,0.0,0.0
3,2019-06-07,DL,DL_CODESHARE,DL,7363,OO,7363,10141,ABR,"Aberdeen, SD",...,73.0,75.0,44.0,1.0,257.0,0.0,0.0,0.0,0.0,0.0
4,2018-07-22,UA,UA,UA,2048,UA,2048,12892,LAX,"Los Angeles, CA",...,75.0,64.0,44.0,1.0,236.0,0.0,0.0,0.0,0.0,0.0


## Type cleaning

First, let's convert date and time columns to datetime format

In [25]:
f = lambda x: datetime.strptime(x, "%Y-%m-%d")

df['fl_date'] = df['fl_date'].apply(f)

In [26]:
timecols = ['crs_dep_time','dep_time','wheels_off','wheels_on','crs_arr_time','arr_time']
# ignoring columns about durations for now

In [27]:
# convert time columns to datetime format
def time_convert(time):
    if np.isnan(time):
        return time
    else:
        s = str(time) # convert to string
        s = s.split(".")[0] # zero-pad
        if len(s) < 4:
            s = "0" * (4-len(s)) + s
        if s.startswith("24"): # datetime can't handle "24", convert to "00"
            s = "00" + s[-2:]
        try:
            s = datetime.strptime(s, "%H%M").time() # convert to datetime format
            return s
        except:
            print("Value failed to convert: ", time)          

In [28]:
for col in timecols:
    df[col] = df[col].apply(time_convert)

In [29]:
# Check it worked
df[timecols].head()

,crs_dep_time,dep_time,wheels_off,wheels_on,crs_arr_time,arr_time
0,07:00:00,06:56:00,07:12:00,08:37:00,09:00:00,08:47:00
1,15:20:00,15:34:00,15:53:00,17:16:00,17:12:00,17:22:00
2,08:01:00,07:57:00,08:11:00,08:56:00,09:20:00,09:12:00
3,13:19:00,13:29:00,13:53:00,14:37:00,14:32:00,14:44:00
4,19:05:00,19:02:00,19:16:00,20:00:00,20:20:00,20:06:00


In [30]:
# # infer remaining dep_delays based on crs_dep_time and dep_time
# # this is messing things up right now, so I'll leave it

# def infer(row):
#     if type(row['dep_delay']) != datetime.time:
#         planned = row['crs_dep_time']
#         actual = row['dep_time']
#         plan = datetime.combine(date.today(), planned)
#         act = datetime.combine(date.today(), actual)
#         delta = plan-act
#         if delta.seconds/(60*60) > 12:
#             act += timedelta(days=1)
#             return (act-plan).seconds / 60
#         elif delta.days < 0:
#             return -delta.seconds/60
#         else:
#             return (act-plan).seconds / 60
#     else:
#         return row['dep_delay']
    
# df['dep_delay'] = df.apply(infer, axis=1)

In [31]:
whatsleft(df)

dep_delay     1
taxi_out      4
wheels_off    4
wheels_on     4
taxi_in       4
air_time      4
dtype: int64

In [32]:
# just drop the remaining null rows
print(len(df))
df = df[df.dep_delay.notnull()]
print(len(df))
df = df[df.taxi_out.notnull()]
print(len(df))

9796
9795
9791


In [33]:
whatsleft(df)

Series([], dtype: float64)

Done! At least for the sample dataset.

### Cancelled flights

In [35]:
# first,drop same columns
df_can = df_can.drop(columns=dropcols)
len(df_can)

170

In [36]:
whatsleft(df_can)

dep_time               161
dep_delay              161
taxi_out               170
wheels_off             170
wheels_on              170
taxi_in                170
arr_time               170
arr_delay              170
actual_elapsed_time    170
air_time               170
carrier_delay          170
weather_delay          170
nas_delay              170
security_delay         170
late_aircraft_delay    170
dtype: int64

What should we do with the rows about flights that got cancelled? On the one hand, they seem like significant events to take into account. On the other hand, they have null values for departure time, delay times etc. and it's hard to know how to fill them in that isn't misleading. That, and they're only about 1.7% of the dataset...

> Currently, the decision is simply to leave them out.

## Other categorical rows - check for bad data

In [53]:
# show layout of columns with type "object"
obj_cols = df.dtypes[df.dtypes == object].index
for col in obj_cols:
    print(col)
    print(df[col].sort_values().unique())

mkt_unique_carrier
['AA' 'AS' 'B6' 'DL' 'F9' 'G4' 'HA' 'NK' 'UA' 'VX' 'WN']
branded_code_share
['AA' 'AA_CODESHARE' 'AS' 'AS_CODESHARE' 'B6' 'DL' 'DL_CODESHARE' 'F9'
 'G4' 'HA' 'HA_CODESHARE' 'NK' 'UA' 'UA_CODESHARE' 'VX' 'WN']
mkt_carrier
['AA' 'AS' 'B6' 'DL' 'F9' 'G4' 'HA' 'NK' 'UA' 'VX' 'WN']
op_unique_carrier
['9E' '9K' 'AA' 'AS' 'AX' 'B6' 'C5' 'CP' 'DL' 'EM' 'EV' 'F9' 'G4' 'G7'
 'HA' 'KS' 'MQ' 'NK' 'OH' 'OO' 'PT' 'QX' 'UA' 'VX' 'WN' 'YV' 'YX' 'ZW']
origin
['ABE' 'ABI' 'ABQ' 'ABR' 'ABY' 'ACK' 'ACT' 'ACV' 'ACY' 'ADQ' 'AEX' 'AGS'
 'ALB' 'ALO' 'AMA' 'ANC' 'APN' 'ART' 'ASE' 'ATL' 'ATW' 'AUS' 'AVL' 'AVP'
 'AZA' 'AZO' 'BDL' 'BET' 'BFL' 'BGM' 'BGR' 'BHM' 'BIL' 'BIS' 'BJI' 'BLI'
 'BMI' 'BNA' 'BOI' 'BOS' 'BPT' 'BQK' 'BQN' 'BRO' 'BTM' 'BTR' 'BTV' 'BUF'
 'BUR' 'BWI' 'BZN' 'CAE' 'CAK' 'CDC' 'CDV' 'CGI' 'CHA' 'CHO' 'CHS' 'CID'
 'CIU' 'CLE' 'CLL' 'CLT' 'CMH' 'CMI' 'CMX' 'COS' 'COU' 'CPR' 'CRP' 'CRW'
 'CSG' 'CVG' 'CWA' 'DAB' 'DAL' 'DAY' 'DBQ' 'DCA' 'DEN' 'DFW' 'DHN' 'DLH'
 'DRO' 'DSM' 'DTW' 'EAT'

No clear need for further cleaning at this point. More columns could be dropped later, but we'll keep them until the feature selection stage.

In [55]:
# write cleaned data to csv file
# df.to_csv('flights_sample_random_cleaned.csv')